In [1]:
# train.py

import os
import torch
import numpy as np
import warnings
import pickle
warnings.filterwarnings(action='ignore')

from data_loader import load_data_1m
from feature_calculations_2 import (
    resample_data, calculate_MA_data, calculate_ema_bollinger_bands, calculate_rsi,
    calculate_macd, calculate_stochastic_oscillator, calculate_adx, calculate_atr,
    calculate_obv, calculate_williams_r, base_feature_fn, cyclic_encode_fn, calculate_support_resistance_numba
)
from strategies import BB_fitness_fn, BB_MACD_fitness_fn
from dataset import make_dataset, replace_nan_with_zero
from train_functions_bi_cul import inference, fitness_fn, generation_valid, generation_test

from Prescriptor import Prescriptor, CryptoModelTCN
from Evolution.crossover import UniformCrossover, WeightedSumCrossover, DifferentialEvolutionOperator, CenDE_DOBLOperator, SkipCrossover
from Evolution.mutation import MultiplyNormalMutation, MultiplyUniformMutation, AddNormalMutation, AddUniformMutation, ChainMutation, FlipSignMutation
from Evolution.mutation import RandomValueMutation
from Evolution.selection import RouletteSelection, TournamentSelection, ParetoLexsortSelection
from Evolution import Evolution

In [2]:
# Load Data
data_1m = load_data_1m('/root/daily/bit/data/1min_bitusdt.pkl')
# data_1m = data_1m.iloc[:200000]

# Resample data to 1D for daily data
data_1d = resample_data(data_1m, '1D')
data_1d['Close time'] = data_1d.index
data_1d = data_1d.reset_index(drop=True)

# 기존 사이클 인코딩 등 (필요에 따라 실행)
data_1d, cyclice_encoding_1d = cyclic_encode_fn(data_1d, 'Close time', 'day_of_week')
data_1m, cyclice_encoding = cyclic_encode_fn(data_1m, 'Open time')

# -------------------------------------------
# 1분봉 데이터: 다양한 윈도우(예: 20, 60, 120, 240, 480봉)를 사용하여 feature 생성
time_windows_1m = [20, 60, 120, 240, 480]
features_1m = {}  # indicator별 컬럼명을 저장할 dictionary

for window in time_windows_1m:
    # 이동평균 (EMA)
    data_1m, ma_cols, ma_cols_rel = calculate_MA_data(data_1m, window, 'EMA')
    features_1m[f'MA_{window}'] = ma_cols_rel

    # EMA Bollinger Bands
    data_1m, bb_cols, bb_cols_rel = calculate_ema_bollinger_bands(data_1m, window)
    features_1m[f'BB_{window}'] = bb_cols_rel

    # RSI (윈도우 크기의 1/3 정도를 사용하거나 최소 5)
    rsi_window = max(5, int(window / 3))
    data_1m, rsi_cols = calculate_rsi(data_1m, window=rsi_window)
    features_1m[f'RSI_{window}'] = rsi_cols

    # MACD: 단기, 장기, 신호선 기간을 window에 비례하여 설정 (예시)
    short_period = max(5, int(window / 4))
    long_period = max(short_period + 5, int(window / 2))
    signal_period = max(3, int(window / 8))
    data_1m, macd_cols = calculate_macd(data_1m, short_period, long_period, signal_period)
    features_1m[f'MACD_{window}'] = macd_cols

    # Stochastic Oscillator
    k_period = max(5, int(window / 4))
    d_period = max(3, int(window / 8))
    data_1m, stoch_cols = calculate_stochastic_oscillator(data_1m, window, k_period, extra_str=f'_{window}')
    features_1m[f'STOCH_{window}'] = stoch_cols

    # ADX
    data_1m, adx_cols = calculate_adx(data_1m, window)
    features_1m[f'ADX_{window}'] = adx_cols

    # ATR
    data_1m, atr_cols = calculate_atr(data_1m, window)
    features_1m[f'ATR_{window}'] = atr_cols

    # OBV
    data_1m, obv_cols = calculate_obv(data_1m)
    features_1m[f'OBV_{window}'] = obv_cols

    # Williams %R
    data_1m, will_cols = calculate_williams_r(data_1m, window)
    features_1m[f'WILL_%R_{window}'] = will_cols

    # Support & Resistance (예시)
    data_1m, sr_col = calculate_support_resistance_numba(data_1m, window=window)
    features_1m[f'SR_{window}'] = sr_col

    # Base Feature (예시: alpha 값을 window에 따라 조정)
    alpha_val = window * 5
    data_1m, base_feature = base_feature_fn(data_1m, alpha=alpha_val)
    features_1m[f'BASE_{window}'] = base_feature

# -------------------------------------------
# 1일봉 데이터: 다양한 윈도우 (예: 20, 60, 120봉)를 사용하여 feature 생성
time_windows_1d = [20, 60, 120]
features_1d = {}

for window in time_windows_1d:
    # 이동평균 (EMA)
    data_1d, ma_cols_1d, ma_cols_rel_1d = calculate_MA_data(data_1d, window, 'EMA', extra_str=f'_{window}')
    features_1d[f'MA_{window}'] = ma_cols_rel_1d

    # EMA Bollinger Bands
    data_1d, bb_cols_1d, bb_cols_rel_1d = calculate_ema_bollinger_bands(data_1d, window, extra_str=f'_{window}')
    features_1d[f'BB_{window}'] = bb_cols_rel_1d

    # RSI (윈도우의 1/3 또는 최소 5)
    rsi_window = max(5, int(window / 3))
    data_1d, rsi_cols_1d = calculate_rsi(data_1d, window=rsi_window, extra_str=f'_{window}')
    features_1d[f'RSI_{window}'] = rsi_cols_1d

    # MACD
    short_period = max(5, int(window / 4))
    long_period = max(short_period + 5, int(window / 2))
    signal_period = max(3, int(window / 8))
    data_1d, macd_cols_1d = calculate_macd(data_1d, short_period, long_period, signal_period, extra_str=f'_{window}')
    features_1d[f'MACD_{window}'] = macd_cols_1d

    # Stochastic Oscillator
    k_period = max(5, int(window / 4))
    d_period = max(3, int(window / 8))
    data_1d, stoch_cols_1d = calculate_stochastic_oscillator(data_1d, window, k_period, extra_str=f'_{window}')
    features_1d[f'STOCH_{window}'] = stoch_cols_1d

    # ADX
    data_1d, adx_cols_1d = calculate_adx(data_1d, window, extra_str=f'_{window}')
    features_1d[f'ADX_{window}'] = adx_cols_1d

    # ATR
    data_1d, atr_cols_1d = calculate_atr(data_1d, window, extra_str=f'_{window}')
    features_1d[f'ATR_{window}'] = atr_cols_1d

    # OBV
    data_1d, obv_cols_1d = calculate_obv(data_1d, extra_str=f'_{window}')
    features_1d[f'OBV_{window}'] = obv_cols_1d

    # Williams %R
    data_1d, will_cols_1d = calculate_williams_r(data_1d, window, extra_str=f'_{window}')
    features_1d[f'WILL_%R_{window}'] = will_cols_1d

    # Support & Resistance
    data_1d, sr_col_1d = calculate_support_resistance_numba(data_1d, window=window, extra_str=f'_{window}')
    features_1d[f'SR_{window}'] = sr_col_1d

    # Base Feature (예시: alpha 값을 window에 따라 조정)
    alpha_val = window * 2
    data_1d, base_feature_1d = base_feature_fn(data_1d, extra_str=f'_{window}', alpha=alpha_val)
    features_1d[f'BASE_{window}'] = base_feature_1d

# -------------------------------------------
# 최종 Feature Column 구성
# 사용하지 않을 컬럼 (드롭할 컬럼) 지정
drop_column = [
    'Open time', 'Close time', 'Quote asset volume', 'Ignore',
    'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume'
]

# 테스트용 컬럼 (예시)
test_column = ['Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume']

# 1분봉 데이터 최종 feature column 구성: 기존 test_column과 각 indicator별 컬럼 리스트를 합산
final_feature_columns_1m = test_column + sum([features_1m[key] for key in features_1m.keys()], [])

# 1일봉 데이터 최종 feature column 구성
final_feature_columns_1d = test_column + sum([features_1d[key] for key in features_1d.keys()], [])

# 이제 data_1m, data_1d와 각각의 final_feature_columns를 모델 입력 등으로 활용할 수 있습니다.


# # bb_entry_pos_list, patience_list, bb_entry_index_list = BB_fitness_fn(data_1m)
# bb_macd_entry_pos_list, patience_list, bb_macd_entry_index_list = BB_MACD_fitness_fn(data_1m, 240, 60, 180)
# # bb_macd_entry_pos_list, patience_list, bb_macd_entry_index_list = BB_MACD_fitness_fn(data_1m, 60, 20, 60)

# # Prepare Dataset
# data_tensor = make_dataset(
#     data_1m, data_1d,
#     using_column=feature_column, using_column_1d=feature_column_1d,
#     window_size=600, window_size_1d=60,
#     entry_pos_list=bb_macd_entry_pos_list, patience_list=patience_list,
#     use_1d_data=True
# )
# entry_pos_list = np.array(bb_macd_entry_pos_list)[np.array(bb_macd_entry_pos_list) != 'hold']

# dataset_1m = []
# dataset_1d = []
# skip_data_cnt = 0
# for data in data_tensor:
#     if len(data[0]) == 600 and len(data[1]) == 60:
#         dataset_1m.append(torch.from_numpy(data[0]).unsqueeze(dim=0))
#         dataset_1d.append(torch.from_numpy(data[1]).unsqueeze(dim=0))
#     else:
#         skip_data_cnt += 1
# dataset_1m = torch.cat(dataset_1m, dim=0)
# dataset_1d = torch.cat(dataset_1d, dim=0)

# # Avoid division by zero by replacing zero denominators with a small epsilon value
# epsilon = 1e-10
# dataset_1m[:, :, :4] = dataset_1m[:, :, :4] / (torch.mean(dataset_1m[:, :, :4], dim=1).unsqueeze(dim=1) + epsilon)
# dataset_1d[:, :, :4] = dataset_1d[:, :, :4] / (torch.mean(dataset_1d[:, :, :4], dim=1).unsqueeze(dim=1) + epsilon)

# dataset_1m = replace_nan_with_zero(dataset_1m)
# dataset_1d = replace_nan_with_zero(dataset_1d)

# import pickle

# data_to_save = {
#     'dataset_1m': dataset_1m,
#     'dataset_1d': dataset_1d,
#     'skip_data_cnt': skip_data_cnt,
#     'entry_pos_list': entry_pos_list,
#     'bb_macd_entry_pos_list': bb_macd_entry_pos_list,
#     'bb_macd_entry_index_list': bb_macd_entry_index_list
# }

# with open('/root/daily/bit_4/backup_feature_data/data.pkl', 'wb') as f:
#     pickle.dump(data_to_save, f)

In [3]:
# Load data from the file into separate variables
with open('/root/daily/bit_4/backup_feature_data/data.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

dataset_1m = loaded_data['dataset_1m']
dataset_1d = loaded_data['dataset_1d']
skip_data_cnt = loaded_data['skip_data_cnt']
entry_pos_list = loaded_data['entry_pos_list']
bb_macd_entry_pos_list = loaded_data['bb_macd_entry_pos_list']
bb_macd_entry_index_list = loaded_data['bb_macd_entry_index_list']

In [4]:
valid_skip_data_cnt = int(len(dataset_1m)*0.6) + skip_data_cnt
test_skip_data_cnt = int(len(dataset_1m)*0.8) + skip_data_cnt


In [ ]:
# Evolution Setup
# 전역적으로 기울기 계산 비활성화
torch.set_grad_enabled(False)
torch.backends.cudnn.benchmark = True

chromosomes_size=10000
window_size=240
EPOCH = 8
gen_loop=50
best_size=10000
elite_size=2000
profit_init=1
device = 'cuda:1'
group = 10000
start_gen = 0
best_profit = None
best_chromosomes = None

# state_dict_path = '/root/daily/bit_4/generation/generation_27.pt'
# if os.path.exists(state_dict_path):
#     state_dict = torch.load(state_dict_path)
#     start_gen = state_dict['generation'] + 1
#     best_profit = state_dict['best_profit']
#     best_chromosomes = state_dict['best_chromosomes']
    # prescriptor.load_state_dict(state_dict['prescriptor_state_dict'],strict=True)

for i in range(EPOCH):
    prescriptor = Prescriptor(
        basic_block=CryptoModelTCN, 
        small_input_dim=25, 
        large_input_dim=25,
        fc_hidden_size=16,
        small_lstm_hidden_dim=16,
        large_lstm_hidden_dim=16,
        output_dim=8, 
        after_input_dim=11, 
        after_hidden_dim=16, 
        after_output_dim=5, 
        num_blocks=group,
    ).to(device).eval()

    # if i == 1:
    #     start_gen=0

    # if i == 0:
    #     prescriptor.load_state_dict(state_dict['prescriptor_state_dict'],strict=True)

    total_param = sum(p.numel() for p in prescriptor.parameters())
    print(f"Total parameters: {total_param}")

    selection = RouletteSelection(elite_num=2000, parents_num=2000, minimize=False)
    # selection = ParetoLexsortSelection(elite_num=2000, parents_num=4000,
    #                                     priority=[], prior_ratio= [],
    #                                     prob_method= 'softmax',minimize=False)
    # crossover = DifferentialEvolutionOperator()
    # crossover = UniformCrossover(num_parents=4)
    # crossover = CenDE_DOBLOperator()
    mutation = ChainMutation([RandomValueMutation(mut_prob=0.05), AddUniformMutation(mut_prob=0.1)])
    crossover = UniformCrossover(num_parents=1)
    # mutation = AddNormalMutation(mut_prob=0.1)
    evolution = Evolution(
        prescriptor=prescriptor,
        selection=selection,
        crossover=crossover,
        mutation=mutation
    )

    best_chromosomes, best_profit = generation_valid(
        data_1m=data_1m,
        dataset_1m=dataset_1m,
        dataset_1d=dataset_1d,
        prescriptor=prescriptor,
        evolution=evolution,
        skip_data_cnt=skip_data_cnt,
        valid_skip_data_cnt=valid_skip_data_cnt,
        test_skip_data_cnt=test_skip_data_cnt,
        chromosomes_size=chromosomes_size,
        window_size=window_size,
        gen_loop=gen_loop,
        best_size=best_size,
        elite_size=elite_size,
        profit_init=profit_init,
        entry_index_list=bb_macd_entry_index_list,
        entry_pos_list=entry_pos_list,
        best_profit=best_profit,
        best_chromosomes=best_chromosomes,
        start_gen=start_gen,
        device=device
    )

Total parameters: 173950000
generation  0: 


 61%|██████    | 25725/42211 [03:17<02:06, 130.52it/s]


generation  1: 


 61%|██████    | 25725/42211 [05:11<03:19, 82.50it/s] 


generation  2: 


...

 80%|████████  | 33968/42211 [01:16<00:18, 442.64it/s]   


generation  16: 


 80%|████████  | 33968/42211 [01:15<00:18, 447.15it/s]   


generation  17: 


 35%|███▌      | 14875/42211 [02:50<05:49, 78.18it/s] 